# Write snapshots

This notebook has the pourpose of exibiting how to write snapshots using various codes.

In [2]:
import numpy as np

# writting/reading libraries
import unsio.output as uns_out
import snapwrite
import h5py

# plotting
from matplotlib import pyplot as plt

Creating distribution of halo particles in order to test snapshot creation.

Unsio can be a bit anoying with data types.
* All quantities need to be set with `dtype = "float32"` in a numpy array.

In [3]:
# Constant distribution inside a box with null velocity

TOTAL_PARTICLES = 10000

positions = np.array(np.random.rand(TOTAL_PARTICLES * 3), dtype="float32") * 100
velocities = np.zeros(TOTAL_PARTICLES * 3, dtype="float32")
masses = np.ones(TOTAL_PARTICLES, dtype="float32")


positions_hdf5 = np.reshape(positions, (TOTAL_PARTICLES, 3))
velocities_hdf5 = np.reshape(velocities, (TOTAL_PARTICLES, 3))


In [5]:
print(f"{positions.shape=}, {positions_hdf5.shape=}")

positions.shape=(30000,), positions_hdf5.shape=(10000, 3)


## Gadget 2

Both aproaches to writing in the binary format, UNSIO and snapwrite, receive positions and velocities as 1-dimensional vectors/lists/arrays with properties sequentially organized as xyzxyz or VxVyVzVxVyVz.

### UNSIO

In [6]:
snapshot_object = uns_out.CUNS_OUT("test_unsio.G2", "gadget2")

# possible components := gas, halo, dm, disk, stars, bulge, bndry

snapshot_object.setData(positions, "halo", "pos")
snapshot_object.setData(velocities, "halo", "vel")
snapshot_object.setData(masses, "halo", "mass")

snapshot_object.save() # essential
snapshot_object.close() # unsio apparently doesn't work with "with" statements. It needs to be manually closed.

### snapwrite

Manual code that delves into directly writing binary. Made originally by Ruggiero, but modified and tweaked by me.

In [9]:

ids = range(TOTAL_PARTICLES)
n_part = [0.0, TOTAL_PARTICLES, 0.0, 0.0, 0.0, 0.0] # list that determines size of each component
data = [positions, velocities, ids, masses] # data list in a way snapwrite understands


snapwrite.write_snapshot(n_part=n_part, data_list=data, outfile="test_snapwrite.G2")

### HDF5

Its HDF5.

Positions and velocities are written as lists of 3-dimensional vectors.

In [44]:
ids = range(TOTAL_PARTICLES)
n_part = [0.0, TOTAL_PARTICLES, 0.0, 0.0, 0.0, 0.0] # list that determines size of each component


with h5py.File("test_hdf5.hdf5", "w") as snapshot_object:
    type1 = snapshot_object.create_group("PartType1")
    
    type1.create_dataset("Coordinates", data=positions_hdf5)
    type1.create_dataset("Velocities", data=velocities_hdf5)
    type1.create_dataset("Masses", data=masses)
    type1.create_dataset("ParticlesIDs", data=ids, dtype='uint32')

    header = snapshot_object.create_group("Header")

    header.attrs['NumPart_ThisFile'] = np.asarray(n_part)
    header.attrs['NumPart_Total'] = np.asarray(n_part)
    header.attrs['NumPart_Total_HighWord'] = 0 * np.asarray(n_part)
    header.attrs['MassTable'] = np.zeros(6)
    header.attrs['Time'] = float(0.0)
    header.attrs['Redshift'] = float(0.0)
    header.attrs['BoxSize'] = float(0.0)
    header.attrs['NumFilesPerSnapshot'] = int(1)
    header.attrs['Omega0'] = float(0.0)
    header.attrs['OmegaLambda'] = float(0.0)
    header.attrs['HubbleParam'] = float(1.0)
    header.attrs['Flag_Sfr'] = int(0.0)
    header.attrs['Flag_Cooling'] = int(0)
    header.attrs['Flag_StellarAge'] = int(0)
    header.attrs['Flag_Metals'] = int(0)
    header.attrs['Flag_Feedback'] = int(0)
    header.attrs['Flag_DoublePrecision'] = 1
    header.attrs['Flag_IC_Info'] = 0